Assurez-vous avant toute chose d'obtenir une clé pour utiliser l'API de Airlabs à l'adresse ci-après : https://airlabs.co/docs/flights

Veuillez configurer un environnement virtuel et y installer l'environnement de développement indiqué dans le fichier requirements.txt

_______________________________________________________________________________________

Les données utilisées sont stockées dans des fichiers au format JSON répartis dans trois collections. 
Les tables suivantes décrivent le schéma défini pour chaque collection de jeux de données : AIRLINES ; DEPARTURES ; FLIGHTS.

Veuillez vous référer au document des spécifications de conception logiciel (2022-10-07-DSTA-Technical_Specifications.odt) pour plus d'informations.

In [1]:
# Importation of Python libraries
from pprint import pprint
import pandas as pd
from IPython.display import display
import requests
import urllib.request
import json
import ssl
from bokeh.plotting import figure, show, ColumnDataSource
from bokeh.tile_providers import STAMEN_TONER
import numpy as np

In [2]:
# Header
ssl._create_default_https_context = ssl._create_unverified_context
opener = urllib.request.build_opener()
opener.addheaders = [('User-agent', 'Mozilla/5.0')]

In [3]:
# Provide your API key in the query string!!!
params = {
  'api_key': '9d2955c1-020a-4343-a6d7-c2f04450735b'
}
end_point = [
    "flights?",
    "airlines?",
    "schedules?"
]

In [7]:
# Real time flight tracking URL:
api_base = 'http://airlabs.co/api/v9/'

# Loading of the data into a JSON named 'flights.json':
api_result = requests.get(api_base+end_point[0], params)
api_response = api_result.json()
api_flights = api_response["response"]

with open("flights.json", "w") as fp:
    json.dump(api_flights, fp, indent=4, sort_keys=True)

# Load the JSON file into a dataframe:
df = pd.read_json("flights.json")
# Control that it is working by printing the dataframe:
#display(df)

#Convert to Pandas Dataframe
flight_df = pd.DataFrame(df, columns = ['lat', 'lng', 'alt', 'flight_icao', 'dep_iata', 'arr_iata', 'airline_iata'])
display(flight_df)


,lat,lng,alt,flight_icao,dep_iata,arr_iata,airline_iata
0,57.654945,59.394657,10668,AFL1460,SVO,OVB,SU
1,55.860397,43.536156,10965,AFL1411,SVX,SVO,SU
2,16.528698,79.044426,10972,AFL289,CMB,SVO,SU
3,43.519501,48.716911,11887,AFL2149,AYT,SVO,SU
4,41.100723,40.510498,11582,AFL2142,SVO,AYT,SU
...,...,...,...,...,...,...,...
7872,35.750000,140.390000,0,FDX5015,NRT,KIX,FX
7873,35.780000,140.390000,0,APJ594,MMB,NRT,MM
7874,42.780000,141.690000,0,APJ110,CTS,KIX,MM
7875,35.780000,140.390000,0,ANA8434,PVG,NRT,NH


Génération d'une carte en se basant sur la méthodologie de Mercator.
_______________________________________________________________________________________

In [ ]:
# Define country coordinate range with Mercator method (EPSG:3857)
x_range,y_range=([-15187814,-6458032], [2505715,6567666])

#Function to convert coordinate to web Mercator
def wgs84_to_web_mercator(df, lon="Long", lat="Lat"):
    k = 6378137
    df["x"] = df[lon] * (k * np.pi/180.0)
    df["y"] = np.log(np.tan((90 + df[lat]) * np.pi/360.0)) * k
    return df
wgs84_to_web_mercator(flight_df)

#Bokeh column data source
flight_source = ColumnDataSource(flight_df)

#Setup figure
p=figure(x_range=x_range,y_range=y_range,x_axis_type='mercator',y_axis_type='mercator',
  sizing_mode='scale_width',plot_height=300)
p.add_tile(STAMEN_TONER)
p.circle('x','y',source=flight_source,fill_color="blue",size=10,fill_alpha=0.8,line_width=0.5)

#Display figure
show(p)